<a href="https://colab.research.google.com/github/heytian/d2d-oco3-tools/blob/main/nc4_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NC4 to CSV - CO2**

This script batch processes netcdf (.nc4) files of OCO-3 CO2 Lite data (from https://oco2.gesdisc.eosdis.nasa.gov/data/OCO3_DATA/OCO3_L2_Lite_FP.11r/) to create a csv containing high quality SAMs tagged as "fossil".

**IMPORTANT: Clear all outputs and end runtime session before saving to Colab or Github to avoid exposing your Earthdata credentials!**

In [ ]:
# Run this to write .netrc to Colab for your Earthdata credentials. RESTART SESSION AND CLEAR OUTPUTS AFTER USE!

import getpass, os

u = getpass.getpass("Earthdata Username: ")
p = getpass.getpass("Earthdata Password: ")

netrc_path = os.path.expanduser("~/.netrc")
with open(netrc_path, "w") as f:
    f.write(f"machine urs.earthdata.nasa.gov\n"
            f"  login {u}\n"
            f"  password {p}\n")

os.chmod(netrc_path, 0o600)

cookie_path = os.path.expanduser("~/.urs_cookies")
open(cookie_path, "a").close()
os.chmod(cookie_path, 0o600)

print("Credentials loaded securely.")


In [ ]:
# !pip install pycountry

In [ ]:
import os
import io
import time
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm
from getpass import getpass
from urllib.parse import urljoin
import requests
from concurrent.futures import ThreadPoolExecutor
import pycountry
from functools import partial
from tqdm import tqdm


BASE_URL = "https://oco2.gesdisc.eosdis.nasa.gov/data/OCO3_DATA/"
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_earthdata_session():
    session = requests.Session()
    netrc_path = os.path.expanduser("~/.netrc")
    if os.path.exists(netrc_path):
        session.trust_env = True
        return session
    username = input("Earthdata username: ")
    password = getpass("Earthdata password: ")
    session.auth = (username, password)
    return session

session = get_earthdata_session()

def list_remote_nc4(product_dir):
    try:
        r = session.get(product_dir)
        r.raise_for_status()
        lines = r.text.splitlines()
        files = [
            L.split('href="')[1].split('"')[0]
            for L in lines
            if ".nc4" in L and not L.strip().endswith('.xml') and 'href' in L
        ]
        return sorted(files)
    except Exception as e:
        return []

def safe_open_h5(bio):
    bio.seek(0)
    sig = bio.read(8)
    bio.seek(0)
    if sig != b'\x89HDF\r\n\x1a\n':
        raise OSError("Not a valid HDF5 file")
    return h5py.File(bio, 'r')

def parse_city_country_pycountry(target_names):
    s = target_names.str.replace('fossil_', '', regex=False)
    parts = s.str.split('_')

    country_names = {c.name.lower() for c in pycountry.countries}
    country_names.update({getattr(c, "common_name", "").lower() for c in pycountry.countries if hasattr(c, "common_name")})

    def get_city_country(lst):
        for n in range(3, 0, -1):
            if len(lst) < n:
                continue
            candidate = ' '.join(lst[-n:]).lower()
            if candidate in country_names:
                city = ' '.join(lst[:-n])
                country = ' '.join(lst[-n:])
                return city, country
        return ' '.join(lst[:-1]), lst[-1]

    city_country = parts.apply(get_city_country)
    city = city_country.apply(lambda x: x[0])
    country = city_country.apply(lambda x: x[1])
    return city, country

def read_filter_remote_file(filename, product_dir, retries=3):
    mapping = {0:b'ND',1:b'GL',2:b'TG',3:b'XS',4:b'AM'}

    for attempt in range(retries):
        try:
            url = urljoin(product_dir, filename)
            with session.get(url, stream=True) as r:
                r.raise_for_status()
                bio = io.BytesIO(r.content)

            with safe_open_h5(bio) as f:
                sid = f['sounding_id'][:]
                xco2 = f['xco2'][:]
                qflag = f['xco2_quality_flag'][:]
                target_name = f['Sounding/target_name'][:]
                lat = f['latitude'][:]
                lon = f['longitude'][:]
                op_mode = f['Sounding/operation_mode'][:]

                op_mode_mapped = np.array([mapping.get(int(v), b'UN') for v in op_mode])

                # Filter for good quality SAMs
                mask = (op_mode_mapped == b'AM') & (qflag == 0)
                sid, xco2, target_name, lat, lon, op_mode_mapped = (
                    sid[mask], xco2[mask], target_name[mask], lat[mask], lon[mask], op_mode_mapped[mask]
                )

                if len(sid) == 0:
                    return None

                target_name_decoded = np.array([t.decode('utf-8').strip() if isinstance(t, bytes) else str(t).strip() for t in target_name])

                fossil_mask = np.char.startswith(target_name_decoded.astype(str), 'fossil_')
                sid, xco2, target_name_decoded, lat, lon, op_mode_mapped = (
                    sid[fossil_mask], xco2[fossil_mask], target_name_decoded[fossil_mask],
                    lat[fossil_mask], lon[fossil_mask], op_mode_mapped[fossil_mask]
                )

                if len(sid) == 0:
                    return None

                df = pd.DataFrame({
                    'sounding_id': sid,
                    'xco2': xco2,
                    'xco2_quality_flag': np.zeros_like(sid, dtype=int),
                    'target_name': target_name_decoded,
                    'latitude': lat,
                    'longitude': lon,
                    'operation_mode': op_mode_mapped
                })

                dt_strings = df['sounding_id'].astype(str).str[:14]
                df['datetime'] = pd.to_datetime(dt_strings, format='%Y%m%d%H%M%S', errors='coerce')

                city, country = parse_city_country_pycountry(df['target_name'])
                df['city'] = city
                df['country'] = country

                return df

        except (requests.HTTPError, OSError, KeyError):
            time.sleep(1)
            continue

    return None

def combine_remote_files_years(years, product_template="OCO3_L2_Lite_FP.11r/{year}/",
                                n_files=None, output_csv="combined_CO2_2019-2025.csv", max_workers=4):
    all_dfs = []

    for year in years:
        print(f"\nProcessing year: {year}")
        product_dir = urljoin(BASE_URL, product_template.format(year=year))
        remote_files = list_remote_nc4(product_dir)
        selected_files = remote_files if n_files is None else remote_files[:n_files]

        dfs = []

        func = partial(read_filter_remote_file, product_dir=product_dir)

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(tqdm(executor.map(lambda f: read_filter_remote_file(f, product_dir),
                                             selected_files), total=len(selected_files)))
            dfs = [r for r in results if r is not None and len(r) > 0]

        if dfs:
            all_dfs.append(pd.concat(dfs, ignore_index=True))

    if not all_dfs:
        print("No data found!")
        return None


    df_all = pd.concat(all_dfs, ignore_index=True)


    agg = df_all.groupby(['target_name', 'datetime'], as_index=False).agg({
        'xco2': 'median',
        'latitude': 'mean',
        'longitude': 'mean',
        'operation_mode': 'first',
        'xco2_quality_flag': 'first',
        'city': 'first',
        'country': 'first'
    })

    out_path = os.path.join(OUTPUT_DIR, output_csv)
    agg.to_csv(out_path, index=False)
    print(f"\nSaved combined multi-year CSV: {out_path}")
    return agg

years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]
combine_remote_files_years(years, n_files=None, max_workers=6)


# **NC4 to CSV - SIF **

This script batch processes netcdf (.nc4) files of OCO-3 SIF Lite data (from https://oco2.gesdisc.eosdis.nasa.gov/data/OCO3_DATA/OCO3_L2_Lite_SIF.11r/) to create a csv containing high quality SAMs.


**IMPORTANT: Clear all outputs and end runtime session before saving to Colab or Github to avoid exposing your Earthdata credentials!**

In [ ]:
# Run this to write .netrc to Colab for your Earthdata credentials. RESTART SESSION AND CLEAR OUTPUTS AFTER USE!

import getpass, os

u = getpass.getpass("Earthdata Username: ")
p = getpass.getpass("Earthdata Password: ")

netrc_path = os.path.expanduser("~/.netrc")
with open(netrc_path, "w") as f:
    f.write(f"machine urs.earthdata.nasa.gov\n"
            f"  login {u}\n"
            f"  password {p}\n")

os.chmod(netrc_path, 0o600)

cookie_path = os.path.expanduser("~/.urs_cookies")
open(cookie_path, "a").close()
os.chmod(cookie_path, 0o600)

print("Credentials loaded securely.")


In [ ]:
# pip install pycountry

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import requests
import pycountry
from tqdm import tqdm
from getpass import getpass
from urllib.parse import urljoin


BASE_URL = "https://oco2.gesdisc.eosdis.nasa.gov/data/OCO3_DATA/"
PRODUCT_TEMPLATE = "OCO3_L2_Lite_SIF.11r/{year}/"
OUTPUT_DIR = "/content/drive/MyDrive/Shortcuts/DATA/output" # replace with your own path
os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_earthdata_session():
    session = requests.Session()
    netrc_path = os.path.expanduser("~/.netrc")
    if os.path.exists(netrc_path):
        session.trust_env = True
        return session
    username = input("Earthdata username: ")
    password = getpass("Earthdata password: ")
    session.auth = (username, password)
    return session

session = get_earthdata_session()

def list_remote_nc4(product_dir):
    r = session.get(product_dir, timeout=60)
    r.raise_for_status()

    files = []
    for line in r.text.splitlines():
        if 'href="' not in line:
            continue

        href = line.split('href="')[1].split('"')[0]

        if href.endswith(".nc4") and not href.endswith(".nc4.xml"):
            files.append(href)

    print(f"Found {len(files)} remote NC4 files.")
    return sorted(files)

def safe_open_h5(bio):
    bio.seek(0)
    sig = bio.read(8)
    bio.seek(0)
    if sig != b'\x89HDF\r\n\x1a\n':
        raise OSError("Not a valid HDF5 file")
    return h5py.File(bio, 'r')

def parse_city_country(df):
    seq = df['sequence_name'].astype(str)
    parts = seq.str.split('_')
    df['city'] = parts.apply(lambda x: ' '.join(x[1:-1]) if len(x) > 2 else '')
    def correct_country(c):
        try:
            return pycountry.countries.lookup(c).name
        except LookupError:
            return c
    df['country'] = parts.apply(lambda x: x[-1] if len(x) > 1 else '')
    return df

def read_filter_remote_file(filename, product_dir, retries=3):
    mapping = {0: b'ND', 1: b'GL', 2: b'TG', 3: b'XS', 4: b'AM'}
    for attempt in range(retries):
        try:
            url = urljoin(product_dir, filename)
            with session.get(url, stream=True) as r:
                r.raise_for_status()
                import io
                bio = io.BytesIO(r.content)

            with safe_open_h5(bio) as f:
                sif = f['Daily_SIF_757nm'][:]
                op_mode_arr = np.atleast_1d(f['Metadata/MeasurementMode'][:])
                qflag = f['SimplyGoodOrBadQualityFlag'][:]
                lat = f['Geolocation/latitude'][:]
                lon = f['Geolocation/longitude'][:]
                tai93 = f['Geolocation/time_tai93'][:]
                seq_idx = f['Sequences/SequencesIndex'][:]
                seq_names_all = f['Sequences/SequencesName'][:]

                op_decoded = np.array([mapping.get(int(v), b'UN') for v in op_mode_arr])

                # Filter for good quality SAMs
                mask = (op_decoded == b'AM') & (qflag == 0)
                if not np.any(mask):
                    return None

                sif = sif[mask]
                op_decoded = op_decoded[mask]
                lat = lat[mask]
                lon = lon[mask]
                tai93 = tai93[mask]
                seq_idx = seq_idx[mask]

                sequence_names = [
                    seq_names_all[i].decode('utf-8') if 0 <= i < len(seq_names_all) else 'UNKNOWN'
                    for i in seq_idx
                ]

                epoch = pd.Timestamp("1993-01-01T00:00:00Z")
                datetime = epoch + pd.to_timedelta(tai93.astype(float), unit="s")

                df = pd.DataFrame({
                    'Daily_SIF_757nm': sif,
                    'latitude': lat,
                    'longitude': lon,
                    'datetime': pd.to_datetime(datetime).tz_localize(None),
                    'measurement_mode': [v.decode('utf-8') for v in op_decoded],
                    'sequence_name': sequence_names
                })

                df = parse_city_country(df)

                print(f"{filename}: kept {len(df)} AM/SAM SIF soundings")
                return df

        except (requests.HTTPError, OSError, KeyError) as e:
            print(f"Warning: {filename} attempt {attempt+1} failed: {e}")
            import time; time.sleep(2**attempt)
            continue

    return None

def combine_remote_files_years(years, n_files=None, max_workers=2, output_csv="combined_SIF_2019-2025.csv"):
    all_dfs = []
    for year in years:
        print(f"\nProcessing year: {year}")
        product_dir = urljoin(BASE_URL, PRODUCT_TEMPLATE.format(year=year))
        remote_files = list_remote_nc4(product_dir)
        selected_files = remote_files if n_files is None else remote_files[:n_files]

        dfs = []
        for f in tqdm(selected_files):
            df = read_filter_remote_file(f, product_dir)
            if df is not None and len(df) > 0:
                dfs.append(df)

        if dfs:
            all_dfs.append(pd.concat(dfs, ignore_index=True))

    if not all_dfs:
        print("No data found!")
        return None

    df_all = pd.concat(all_dfs, ignore_index=True)
    out_path = os.path.join(OUTPUT_DIR, output_csv)
    df_all.to_csv(out_path, index=False)
    print(f"\nSaved combined SIF all-values CSV: {out_path}")
    return df_all

years = [2019,2020,2022,2023,2024,2025]
combine_remote_files_years(years, n_files=None, max_workers=2)
